In [21]:
# import library
import pandas as pd

In [22]:
# read the data
data = pd.read_excel("Credit Card Payment Fraud.xlsx")

In [23]:
data.dropna()

,recordnum,cardnum,date,merchnum,merch.description,merch.state,merch.zip,transtype,amount,fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
2,3,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
3,4,5142152067,2010-01-01,5509006296254,FEDEX SHP 12/21/09 AB#,TN,38118.0,P,3.67,0
4,5,5142132941,2010-01-01,5509006296254,FEDEX SHP 12/27/09 AB#,TN,38118.0,P,3.62,0
...,...,...,...,...,...,...,...,...,...,...
95002,95003,5142276099,2010-12-31,602608969138,FISHER SCI CHI,IL,60143.0,P,114.09,0
95003,95004,5142276099,2010-12-31,602608969534,FISHER SCI ATL,GA,30091.0,P,208.36,0
95004,95005,5142289546,2010-12-31,580000003,TEXACO 22601220094,AL,36043.0,P,194.26,0
95005,95006,5142291592,2010-12-31,4503057341100,"TECH PAC, INC",OH,45150.0,P,493.76,0


In [24]:
# clean & transform data types
data[["recordnum"]] = data[["recordnum"]].astype('category')
data[["cardnum"]] = data[["cardnum"]].astype('category')
data[["date"]] = data[["date"]].astype('datetime64[ns]')
data[["merchnum"]] = data[["merchnum"]].astype('category')
data[["merch.description"]] = data[["merch.description"]].astype('str')
data[["merch.state"]] = data[["merch.state"]].astype('category')
data[["merch.zip"]] = data[["merch.zip"]].astype('category')
data[["transtype"]] = data[["transtype"]].astype('category')
data[["amount"]] = data[["amount"]].astype('float')
data[["fraud"]] = data[["fraud"]].astype('category')

In [25]:
# Handle and Impute Missing Values
## Assume Each Merchant will have unique 
## (merchnum, merch.zip, merch.state, merch.description)

In [26]:
## Round 1: 
## Replace the missing row with non-missing rows based on same
## (merchnum, merch.zip, merch.state, merch.description) any three of combo

In [27]:
## merch.zip
merchzip=data.loc[(data["merchnum"].notnull())&(data["merch.description"].notnull())&(data["merch.state"].notnull())&(data["merch.zip"].notnull()),
                  ["merchnum","merch.description","merch.state","merch.zip"]].drop_duplicates()
merchzip_miss=data.loc[data["merch.zip"].isnull(),]
merchzip_nomiss=data.loc[data["merch.zip"].notnull(),]

merchzip_miss_fix=pd.merge(merchzip,merchzip_miss,how='right',
               left_on=["merch.description","merch.state","merchnum"],
               right_on=["merch.description","merch.state","merchnum"])

merchzip_miss_fix=merchzip_miss_fix.iloc[:,[0,1,2,3,4,5,6,8,9,10]]
merchzip_miss_fix.rename(columns={'merch.zip_x':'merch.zip'}, inplace=True)

data=pd.concat([merchzip_nomiss,merchzip_miss_fix])

In [28]:
## merchnum
merchnum=data.loc[(data["merchnum"].notnull())&(data["merch.description"].notnull())&(data["merch.state"].notnull())&(data["merch.zip"].notnull()),
                  ["merchnum","merch.description","merch.state","merch.zip"]].drop_duplicates()
merchnum_miss=data.loc[data["merchnum"].isnull(),]
merchum_nomiss=data.loc[data["merchnum"].notnull(),]

merchnum_miss_fix=pd.merge(merchnum,merchnum_miss,how='right',
               left_on=["merch.description","merch.state","merch.zip"],
               right_on=["merch.description","merch.state","merch.zip"])

merchnum_miss_fix=merchnum_miss_fix.iloc[:,[0,1,2,3,4,5,6,7,9,10]]
merchnum_miss_fix.rename(columns={'merchnum_x':'merchnum'}, inplace=True)

data=pd.concat([merchum_nomiss,merchnum_miss_fix])

In [29]:
## merchstate
merchstate=data.loc[(data["merchnum"].notnull())&(data["merch.description"].notnull())&(data["merch.state"].notnull())&(data["merch.zip"].notnull()),
                  ["merchnum","merch.description","merch.state","merch.zip"]].drop_duplicates()
merchstate_miss=data.loc[data["merch.state"].isnull(),]
merchstate_nomiss=data.loc[data["merch.state"].notnull(),]

merchstate_miss_fix=pd.merge(merchstate,merchstate_miss,how='right',
               left_on=["merch.description","merchnum","merch.zip"],
               right_on=["merch.description","merchnum","merch.zip"])

merchstate_miss_fix=merchstate_miss_fix.iloc[:,[0,1,2,3,4,5,6,7,9,10]]
merchstate_miss_fix.rename(columns={'merch.state_x':'merch.state'}, inplace=True)

data=pd.concat([merchum_nomiss,merchnum_miss_fix])

In [30]:
## Round 2: 
## Assign new value for the rest of missing rows based on same group of
## (merchnum, merch.zip, merch.state, merch.description) any three of combo

In [31]:
## merchzip
merchzip_miss_x=data.loc[data["merch.zip"].isnull(),]
merchzip_nomiss_x=data.loc[data["merch.zip"].notnull(),]
merchdescrip=merchzip_miss_x[["merch.description"]].drop_duplicates()
merchdescrip["merch.zip"]=range(1,497)
merchdescrip["merch.zip"]='M '+merchdescrip["merch.zip"].astype('str')

merchzip_miss_fix_x=pd.merge(merchzip_miss_x,merchdescrip, how='left',
                             left_on="merch.description",
                             right_on="merch.description")

merchzip_miss_fix_x=merchzip_miss_fix_x.iloc[:,[0,1,2,3,4,5,7,8,9,10]]
merchzip_miss_fix_x.rename(columns={'merch.zip_y':'merch.zip'}, inplace=True)


data = pd.concat([merchzip_nomiss_x,merchzip_miss_fix_x])

In [32]:
## merchnum
merchnum_miss_x=data.loc[data["merchnum"].isnull(),]
merchnum_nomiss_x=data.loc[data["merchnum"].notnull(),]
merchdescrip=merchnum_miss_x[["merch.description"]].drop_duplicates()
merchdescrip["merchnum"]=range(1,578)
merchdescrip["merchnum"]='M '+merchdescrip["merchnum"].astype('str')

merchnum_miss_fix_x=pd.merge(merchnum_miss_x,merchdescrip, how='left',
                             left_on="merch.description",
                             right_on="merch.description")

merchnum_miss_fix_x=merchnum_miss_fix_x.iloc[:,[0,1,2,3,4,5,6,8,9,10]]
merchnum_miss_fix_x.rename(columns={'merchnum_y':'merchnum'}, inplace=True)


data = pd.concat([merchnum_nomiss_x,merchnum_miss_fix_x])

In [33]:
## merchstate
merchstate_miss_x=data.loc[data["merch.state"].isnull(),]
merchstate_nomiss_x=data.loc[data["merch.state"].notnull(),]
merchdescrip=merchstate_miss_x[["merch.description"]].drop_duplicates()
merchdescrip["merch.state"]=range(1,152)
merchdescrip["merch.state"]='M '+merchdescrip["merch.state"].astype('str')

merchstate_miss_fix_x=pd.merge(merchstate_miss_x,merchdescrip, how='left',
                             left_on="merch.description",
                             right_on="merch.description")

merchstate_miss_fix_x=merchstate_miss_fix_x.iloc[:,[0,1,2,3,4,6,7,8,9,10]]
merchstate_miss_fix_x.rename(columns={'merch.state_y':'merch.state'}, inplace=True)

data = pd.concat([merchstate_nomiss_x,merchstate_miss_fix_x])

In [34]:
data.dropna()

,recordnum,cardnum,date,merchnum,merch.description,merch.state,merch.zip,transtype,amount,fraud,merchnum_y,merchnum_x


In [ ]:
data.to_excel("Credit Card Payment Fraud Clean.xlsx",index=False)